In [ ]:
import networkx as nx
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

print("Loading P8_bold_peak.qasm...")
qc = QuantumCircuit.from_qasm_file("P8_bold_peak.qasm")

# Identify the Independent Groups
print("Splitting the circuit into components...")
qubit_map = {bit: i for i, bit in enumerate(qc.qubits)}
G = nx.Graph()
G.add_nodes_from(range(qc.num_qubits))

# Build interaction graph
for instruction in qc.data:
    if instruction.qubits and len(instruction.qubits) > 1:
        indices = [qubit_map[q] for q in instruction.qubits]
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                G.add_edge(indices[i], indices[j])

components = list(nx.connected_components(G))
groups = [sorted(list(g)) for g in components]

print(f"Found {len(groups)} independent parts.")
for i, g in enumerate(groups):
    print(f"  - Part {i+1}: {len(g)} qubits")

# Solve Each Part Separately
final_bit_array = ['0'] * qc.num_qubits
sim = AerSimulator(method='statevector')

print("\nSimulating parts...")
for group_idx, group_qubits in enumerate(groups):
    # Create Sub-Circuit
    sub_qc = QuantumCircuit(len(group_qubits))

    # Map Global Qubit Index -> Local Sub-Circuit Index
    global_to_local = {global_idx: local_idx for local_idx, global_idx in enumerate(group_qubits)}

    for instruction in qc.data:
        # Check if gate belongs to this group (by checking first qubit)
        if instruction.qubits and qubit_map[instruction.qubits[0]] in group_qubits:
            op = instruction.operation
            if op.name != 'measure':
                new_qargs = [sub_qc.qubits[global_to_local[qubit_map[q]]] for q in instruction.qubits]
                sub_qc.append(op, new_qargs)

    sub_qc.measure_all()
    result = sim.run(sub_qc, shots=1024).result()
    counts = result.get_counts()
    sub_winner = max(counts, key=counts.get)

    # d. Map Result back to Global Array
    # Reverse because Qiskit string is Little-Endian (Local qLast...q0)
    bits_ordered = list(reversed(sub_winner))

    for local_idx, bit_val in enumerate(bits_ordered):
        global_idx = group_qubits[local_idx]
        final_bit_array[global_idx] = bit_val

# 4. Stitch Final Solution
solution = "".join(reversed(final_bit_array))

print(f"\nSUCCESS!")
print(f"Solution Bitstring: {solution}")

Loading P8_bold_peak.qasm...
Splitting the circuit into components...
Found 3 independent parts.
  - Part 1: 24 qubits
  - Part 2: 26 qubits
  - Part 3: 22 qubits

Simulating parts...

SUCCESS!
Solution Bitstring: 101111100101111000100010110001011101110011100011110000100111010010001010
